In [ ]:
!pip install nb_black
%load_ext nb_black

# Introduction

The purpose of the following exercises is to gain familiarity with some core concepts underlying linear regression (as well as more sophisticated machine learning algorithms). You'll practice implementing gradient descent, coding models with the sklearn library, engineering features, and regularising. The dataset you'll use is [from Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques) and describes houses that were sold in Ames, Iowa. The modelling objective is to use available features to predict the sale price of a house.

In [ ]:
# Packages and functions you may find useful
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load the housing dataset
data = pd.read_csv("../housing.csv")

# Explore and preprocess the data

**Exercise:** Spend a little time exploring the dataset. What features do you think could be predictive of sale price? Are there potential issues to flag, e.g. missing data?

**Exercise:** Make a scatterplot of `SalePrice` vs. `LotArea`. Does there appear to be a linear relationship? Is there a transformation we can apply to make the data appear more linear?

In [ ]:
data = data.assign(
    LogLotArea=data.LotArea.apply(np.log),
    LogSalePrice=data.SalePrice.apply(np.log),
)

plt.figure()
plt.scatter(data.LogLotArea, data.LogSalePrice)
plt.title("Housing Prices in Ames, Iowa")
plt.xlabel("log lot area (square feet)")
plt.ylabel("log sale price (dollars)")
plt.show()

**Exercise:** Divide the data into training and test sets. A train-test split of at least 90-10 is recommended. You may want to further divide the data into 80-10-10 training-validation-test sets if you'd like to tune hyperparameters or select the best model in the end.

In [ ]:
features = [col for col in data.columns if "SalePrice" not in col]
X = data[features]
Y = data.LogSalePrice

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=42
)

# Regression on one variable

For ease of demonstration, we'll start by just using log `LotArea` to predict log `SalePrice`. As you saw above, the dataset offers many more features that could be useful, so a regression in just one variable may not be the best performing. Towards the end of the exercises, you'll have an opportunity to develop your own regression model. 

## Gradient Descent

For a first pass, let's fit a regression model without bias term, i.e.
$$log(SalePrice) = \theta * log(LogLotArea)$$

**Exercise:** In the cell below, define the MSE loss function, the gradient of the loss function with respect to the model parameter $\theta$, and a function that performs gradient descent. The type hints and docstrings suggest a way to implement these functions, but feel free to write them your own way. 

In [ ]:
def loss_fcn(y: pd.Series, x: pd.Series, theta: float) -> float:
    """MSE for univariate linear regression without bias term.
    Parameters
    ----------
    y: pd.Series
        The ground truth.
    x: pd.Series
        Feature observations.
    theta: float
        The regression parameter.
    Returns
    -------
    float
        The MSE for the regression on the input data.
    """
    return np.sum((y - x * theta) ** 2) / len(y)


def grad_loss_fcn(y: pd.Series, x: pd.Series, theta: float) -> float:
    """Gradient of the MSE loss function with respect to theta."""
    return -2 * np.sum(x * (y - theta * x)) / len(y)


def gradient_descent(
    y: pd.Series,
    x: pd.Series,
    init: float,
    alpha: float,
    steps: int,
) -> list:
    """Gradient descent algorithm for a univariate linear regression without bias.
    Parameters
    ----------
    y: pd.Series
        The ground truth.
    x: pd.Series
        Feature observations.
    init: float
        Initial value of the regression parameter.
    alpha: float
        Learning rate.
    steps: int
        Number of iterations of the algorithm.
    Returns
    -------
    list
        List of updated values for theta from gradient descent.
    """
    theta = init
    descent = [theta]

    for n in range(steps):
        theta -= alpha * grad_loss_fcn(y, x, theta)
        descent.append(theta)

    return descent

**Exercise:** Using gradient descent function you defined above, find the optimal value for $\theta$. You may have to experiment with the learning rate and number of steps in order to achieve convergence. Try initialising $\theta$ at different values. Plot the loss function and plot the loss at each stage of the gradient descent.  

In [ ]:
theta = np.linspace(-30, 30, 100)
loss_graph = np.vectorize(lambda theta: loss_fcn(Y_train, X_train.LogLotArea, theta))(
    theta
)

descent = np.array(gradient_descent(Y_train, X_train.LogLotArea, -25, 10 ** (-3), 100))
descent_values = np.vectorize(
    lambda theta: loss_fcn(Y_train, X_train.LogLotArea, theta)
)(descent)

plt.figure()
plt.plot(theta, loss_graph, label="loss function")
plt.scatter(descent, descent_values, c="#F55C5C", label="descent values")
plt.title("Gradient descent")
plt.xlabel("theta")
plt.ylabel("MSE loss")
plt.legend()
plt.show()

**Exercise** Plot the regression you optimised by gradient descent against the data. Evaluate the performance of your model by a metric of your choice.

In [ ]:
x = np.linspace(7, 13, 100)
regression_graph = descent[-1] * x

plt.figure()
plt.scatter(X_train.LogLotArea, Y_train, label="train data")
plt.scatter(
    X_test.LogLotArea, Y_test, facecolors="none", edgecolors="black", label="test data"
)
plt.plot(x, regression_graph, color="#F55C5C", label="regression")
plt.title("Housing Prices in Ames, Iowa")
plt.xlabel("log lot area (square feet)")
plt.ylabel("log sale price (dollars)")
plt.legend()
plt.show()

In [ ]:
# Evaluate performance by MSE
train_mse = loss_fcn(Y_train, X_train.LogLotArea, descent[-1])
test_mse = loss_fcn(Y_test, X_test.LogLotArea, descent[-1])
train_mse, test_mse

## Sklearn implementation

It's great that you can implement linear regression from scratch, but in practice, you'll probably use a library like `sklearn`. The process of fitting and making predictions is essentially the same with `sklearn` no matter what the modelling task is.
1. Instantiate the model class with whatever hyperparameters you choose.
2. Call the fit method on the training data.
3. Call the predict method on the test data.

When in doubt about how to use anything in `sklearn`, refer to the documentation or google your question!

**Exercise**: Use the `LinearRegression` class from `sklearn` to regress log `SalePrice` onto log `LotArea`, and compare the model to the one you developed above. Note that default in `sklearn` is to fit a bias term.

In [ ]:
regression = LinearRegression()
regression.fit(X_train.LogLotArea.values.reshape(-1, 1), Y_train)

x = np.linspace(7, 13, 100)
preds = regression.coef_ * x + regression.intercept_

plt.figure()
plt.scatter(X_train.LogLotArea, Y_train, label="train data")
plt.scatter(
    X_test.LogLotArea, Y_test, facecolors="none", edgecolors="black", label="test data"
)
plt.plot(x, preds, color="#F55C5C", label="regression")
plt.title("Housing Prices in Ames, Iowa")
plt.xlabel("log lot area (square feet)")
plt.ylabel("log sale price (dollars)")
plt.legend()
plt.show()

In [ ]:
train_mse = mean_squared_error(
    Y_train, regression.predict(X_train.LogLotArea.values.reshape(-1, 1))
)
test_mse = mean_squared_error(
    Y_test, regression.predict(X_test.LogLotArea.values.reshape(-1, 1))
)
train_mse, test_mse

**Exercise:** In training the model with an MSE loss function, we assume that the residuals are normally distributed. Assess whether this is a reasonable assumption for our data.

In [ ]:
y_pred = regression.predict(X_train.LogLotArea.values.reshape(-1, 1))
residuals = Y_train - y_pred

normal = norm(loc=0, scale=residuals.std())
x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)

plt.figure()
plt.hist(residuals, bins=20, density=True, label="train residuals")
plt.plot(x, normal.pdf(x), label="normal dist", c="#F55C5C", linewidth=1.5)
plt.title("Prediction error on training set")
plt.xlabel("error")
plt.legend()
plt.show()

## Polynomial regression

Let's see if we can get a better fit by introducing higher powers of log `LotArea` as features. The form of the regression will then be
$$ log(SalePrice) = \theta_0 + \theta_1 * log(LotArea) + \dots + \theta_n * log(LotArea)^n$$
for some $n \geq 1$. With higher order features, there's a risk of overfitting the model to the training data, so we should consider using a regularisation method. 

**Exercise:** Engineer polynomial features in log `LotArea` and fit a linear regression. When working with multiple features, it's good practice to scale them before fitting a model. You may want to use the `Ridge` or `Lasso` class from `sklearn` to regularise the fit and then experiment with the regularisation parameter. Compare the contribution of each feature to the model prediction. Evaluate your model's performance, comparing to the performance of the model you developed above.

In [ ]:
# Scale log LotArea and add higher order features


def add_poly_features(data: pd.DataFrame, feature: str, max_power: int) -> pd.DataFrame:
    df = data.copy()
    for n in range(2, max_power + 1):
        df.loc[:, f"{feature}{n}"] = df[feature] ** n
    return df


MAX_POWER = 9

scaler = MinMaxScaler()

x_train = add_poly_features(X_train, "LogLotArea", MAX_POWER)
x_train = x_train[[col for col in x_train if "LogLotArea" in col]]
x_train = scaler.fit_transform(x_train)

x_test = add_poly_features(X_test, "LogLotArea", MAX_POWER)
x_test = x_test[[col for col in x_test if "LogLotArea" in col]]
x_test = scaler.transform(x_test)

In [ ]:
# Fit regression with varying levels of L2 regularisation.
regression_dict = {}
for i in range(-5, 6):
    regression = Ridge(alpha=10**i)
    regression.fit(x_train, Y_train)
    regression_dict[i] = regression

In [ ]:
x = pd.DataFrame({"LogLotArea": np.linspace(7, 13, 100)})
x = add_poly_features(x, "LogLotArea", MAX_POWER)
x = scaler.transform(x)

plt.figure(figsize=(8, 8))
plt.scatter(x_train[:, 0], Y_train, label="train data")
plt.scatter(
    x_test[:, 0], Y_test, facecolors="none", edgecolors="black", label="test data"
)

for i, regression in regression_dict.items():
    preds = regression.intercept_
    for n in range(0, MAX_POWER):
        preds += regression.coef_[n] * x[:, n]
    plt.plot(x[:, 0], preds, label=f"alpha=10^{i}")

plt.title("Housing Prices in Ames, Iowa")
plt.xlabel("log lot area (square feet)")
plt.ylabel("log sale price (dollars)")
plt.legend()
plt.show()

In [ ]:
regression = regression_dict[-5]

train_mse = mean_squared_error(
    Y_train,
    regression.predict(x_train),
)
test_mse = mean_squared_error(
    Y_test,
    regression.predict(x_test),
)
train_mse, test_mse

# Extension: develop your own regression model

**Exercise:** Now that we've pretty much exhausted the modelling possibilities with just one feature, develop your own regression model with all the features in the dataset at your disposal. Compare the performance of your model to the peformance of the models we produced above. Keep in mind the following tips as you train and evaluate your model.
* Consider one-hot-encoding categorical features.
* Features on different scales should be transformed so that they're comparable.
* If tuning a hyperparameter like the regularisation parameter, it's best practice to further divide your training set into training and validation sets or to cross validate.